# The code given by tutor results in an error when more than 1 city name is entered.

# This code aims to solve that by giving proper prices for all the given cities

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr


In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
# we'll try to impliment methods handle multi inputs in the query
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    #return_prices = []
    #for city in destination_city:
    city = destination_city.lower()
    #return_prices.append(ticket_prices.get(city,"unknown"))
    return ticket_prices.get(city,"Unknown")

In [ ]:
get_ticket_price("paris")

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "List of cities that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(f"response ----------------- \n {response}")
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(f"message: -----------------\n",message)
        response, city = handle_tool_call(message)
        # print('response is --------', response)
        # print('city is ----------',city)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    responses = []
    all_cities = []
    for tool_call in message.tool_calls:
        
        arguments = json.loads(tool_call.function.arguments)
        list_of_city = arguments.get('destination_city')
        print(f'list of city is ======== {list_of_city}')
        price = get_ticket_price(list_of_city)
        print(f'price of ticket to {list_of_city} is {price}')
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": list_of_city,"price": price}),
            "tool_call_id": tool_call.id
        }
        responses.append(response)
        all_cities.append(list_of_city)
    print(f'responses ====== {responses}')
    print(f'cities ======= {all_cities}')
    return responses,all_cities

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()